In [1]:
!pip install tensorflow-macos tensorflow-metal opencv-python mediapipe-silicon scikit-learn matplotlib

You should consider upgrading via the '/Users/rakazein/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from keras.models import load_model

objc[34826]: Class CaptureDelegate is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x14c7b25a0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x12ef48860). One of the two will be used. Which one is undefined.
objc[34826]: Class CVWindow is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x14c7b25f0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11f3f0a68). One of the two will be used. Which one is undefined.
objc[34826]: Class CVView is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x14c7b2618) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11f3f0a90). One of the two will be used. Which one i

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    #Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [8]:
model = load_model('action_v1-20.h5')

Metal device set to: Apple M1 Pro


2023-08-29 14:25:08.028156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-29 14:25:08.028605: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [10]:
from scipy import stats

In [11]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [12]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA_V2') 

# Actions that we try to detect
actions = np.array(['a', 'b', 'c', 
                    'd', 'e', 'f', 
                    'g', 'h', 'i', 
                    'j', 'k', 'l', 
                    'm', 'n', 'o', 
                    'p', 'q', 'r', 
                    's', 't', 'u',
                    'v', 'w', 'x',
                    'y', 'z'
                   ])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [13]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        cv2.rectangle(image, (410, 150, 600, 700) , (0, 255, 0), 1)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if res[np.argmax(res)] >= threshold: 
                print(actions[np.argmax(res)])
                cv2.putText(image, actions[np.argmax(res)], (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 2,  cv2.LINE_AA)    
                sequence = sequence[-30:]
            elif res[np.argmax(res)] < threshold:
                print(actions[np.argmax(res)])
                cv2.putText(image, "Awaiting Gesture", (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 2,  cv2.LINE_AA)    
                sequence = sequence[-30:]
                
            
#         #3. Viz logic
#             if np.unique(predictions[-10:])[0]==np.argmax(res): 
#                 if res[np.argmax(res)] > threshold: 
                    
#                     if len(sentence) > 0: 
#                         if actions[np.argmax(res)] != sentence[-1]:
#                             sentence.append(actions[np.argmax(res)])
#                     else:
#                         sentence.append(actions[np.argmax(res)])

#             if len(sentence) > 5: 
#                 sentence = sentence[-5:]

#             # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
#         cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
#         cv2.putText(image, ' '.join(sentence), (3,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

2023-08-29 14:25:12.620710: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-29 14:25:12.774703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 505ms/step
p
p
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 208ms/step
p
p
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 215ms/step
p
p
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 199ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 201ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 204ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 199ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 205ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 194ms/step
a
a
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 223ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 196ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 196ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 202ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 200ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 198ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 198ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 197ms/step
a
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 204ms/step
a
a
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 199ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 203ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 200ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 211ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 199ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 197ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 207ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 207ms/step
j
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 198ms/step
j
j
<class 'mediapipe.python.soluti